In [2]:
%pip install google-generativeai

  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/155.4 kB ? eta -:--:--
   ---------------------------------------  153.6/155.4 kB 9.0 MB/s eta 0:00:01
   ---------------------------------------- 155.4/155.4 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.3 MB ? eta -:--:--
   -------------------- ------------------- 0.7/1.3 MB 8.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.2/1.3 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/160.8 kB ? eta -:--:--
   --------------------------------------- 160.8/160.8 kB 10.0 MB/s eta 0:00:00
   --------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.5 which is incompatible.


In [16]:
import google.generativeai as genai
import os
from IPython.display import Markdown # Optional: for pretty printing Markdown output


In [18]:
sishishira = "AIzaSyAT3PPOZzhXn4h2Lcs9vMRDsqhMf" # <<< REPLACE WITH YOUR KEY!

# Configure and list available models
genai.configure(api_key=sishishira)

print("Available Gemini Models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"- {m.name} (Description: {m.description})")

# Choose a model
model_name = 'gemini-1.5-flash'
gemini_model = genai.GenerativeModel(model_name)
print(f"\nUsing model: {model_name}")

Available Gemini Models:
- models/gemini-1.0-pro-vision-latest (Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.)
- models/gemini-pro-vision (Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.)
- models/gemini-1.5-pro-latest (Description: Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens.)
- models/gemini-1.5-pro-002 (Description: Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in September of 2024.)
- models/gemini-1.5-pro (Description: Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million to

In [19]:

# --- Function to Get Gemini Response ---
def get_gemini_response_for_exam(prompt_text: str, max_words: int = 1000, model_instance=None) -> str:
    """
    Generates a concise Gemini response for exam-style prompts.

    Args:
        prompt_text (str): The exam question or prompt.
        max_words (int): Maximum number of words allowed in the response.
        model_instance: Pre-initialized Gemini model.

    Returns:
        str: AI-generated answer or error message.
    """
    if model_instance is None:
        return "Error: Gemini model not provided."

    full_prompt = f"{prompt_text}. Generate the response within {max_words} words."
    print(f"\n--- Sending Prompt to Gemini Model ---\n{full_prompt}\n")

    try:
        response = model_instance.generate_content(full_prompt)
        generated_text = getattr(response, 'text', '') or (
            response.candidates[0].text if response.candidates else ''
        )

        if not generated_text:
            return "Error: No valid response from Gemini."

        words = generated_text.split()
        if len(words) > max_words:
            print(f"--- Warning: Response Exceeded Word Limit ({len(words)} words) ---")
            return " ".join(words[:max_words]) + "\n\n[... Response truncated ...]"
        
        print(f"--- Gemini Response Received ({len(words)} words) ---")
        return generated_text

    except Exception as e:
        return f"An error occurred during AI response generation: {e}"

# --- Exam Question Usage ---
exam_prompt = (
    "What is an activation function in a neural network? "
    "Explain its purpose, common types (ReLU, Sigmoid, Tanh), and why non-linearity is important."
)
desired_word_limit = 500

print(f"\n--- Processing Exam Question ---\nQuestion: {exam_prompt}\nWord Limit: {desired_word_limit}")

exam_answer = get_gemini_response_for_exam(
    prompt_text=exam_prompt,
    max_words=desired_word_limit,
    model_instance=gemini_model
)

print("\n--- FINAL EXAM ANSWER ---")
display(Markdown(exam_answer))
print("\n--- End of Response ---")


--- Processing Exam Question ---
Question: What is an activation function in a neural network? Explain its purpose, common types (ReLU, Sigmoid, Tanh), and why non-linearity is important.
Word Limit: 500

--- Sending Prompt to Gemini Model ---
What is an activation function in a neural network? Explain its purpose, common types (ReLU, Sigmoid, Tanh), and why non-linearity is important.. Generate the response within 500 words.

--- Gemini Response Received (442 words) ---

--- FINAL EXAM ANSWER ---


In a neural network, an activation function is a crucial component that introduces non-linearity into the network's output.  Without activation functions, each layer would simply perform a linear transformation on the input, and no matter how many layers you stack, the entire network would effectively be just one linear transformation. This severely limits the network's ability to learn complex patterns in data.

The purpose of an activation function is to transform the linear output of a neuron into a non-linear output. This non-linear transformation allows the network to learn and approximate arbitrarily complex functions, enabling it to solve a wide range of problems.  The output of the activation function then becomes the input for the next layer.

Several common activation functions are used:

* **Sigmoid:**  This function outputs a value between 0 and 1, often interpreted as a probability. Its formula is σ(x) = 1 / (1 + exp(-x)).  While historically popular, sigmoid suffers from the vanishing gradient problem, where gradients become extremely small during backpropagation, hindering training, especially in deep networks.

* **Tanh (Hyperbolic Tangent):**  Similar to sigmoid, tanh outputs a value between -1 and 1.  Its formula is tanh(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x)).  Tanh generally performs better than sigmoid because its output is centered around 0, which can speed up training. However, it also suffers from the vanishing gradient problem.

* **ReLU (Rectified Linear Unit):**  ReLU is a very popular choice due to its simplicity and effectiveness.  It outputs x if x > 0, and 0 otherwise.  Its formula is max(0, x).  ReLU avoids the vanishing gradient problem for positive inputs and is computationally efficient. However, it suffers from the "dying ReLU" problem, where neurons can become inactive if their weights are updated such that the input is always negative, thus preventing them from ever activating.

Why is non-linearity important?

Non-linearity is essential because it allows the network to learn complex relationships between inputs and outputs that cannot be represented by a linear model.  Consider a simple XOR problem:

* Input (0, 0) -> Output 0
* Input (0, 1) -> Output 1
* Input (1, 0) -> Output 1
* Input (1, 1) -> Output 0

This problem is not linearly separable. A linear model cannot draw a line to perfectly separate the inputs that result in 0 from those that result in 1. However, a neural network with a non-linear activation function can easily learn this relationship.  The non-linear activation function allows the network to create decision boundaries that are not straight lines, enabling it to solve non-linearly separable problems.  In essence, non-linearity is what gives neural networks their power to approximate complex functions.



--- End of Response ---
